In [1]:
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import branca
import folium
import json

# Dataset Description

Respondents were asked, "At any time in the past 12 months, did you seriously think about trying to kill yourself?" If they answered "Yes," they were categorized as having serious thoughts of suicide in the past year For substate region definitions, see the "2012-2014 National Survey on Drug Use and Health Substate Region Definitions" at http://www.samhsa.gov/data/. "

Substate, state, and census region estimates along with the 95 percent confidence (credible) intervals are based on a survey-weighted hierarchical Bayes estimation approach and are generated by Markov Chain Monte Carlo techniques. For the "Total U.S." row, design-based (direct) estimates and corresponding 95 percent confidence intervals are given.

The column labeled "Order" can be used to sort the data to the original sort order. The column labeled "Map Group" can be used to distinguish substate regions included in the maps from other substate regions.
In the "Map Group" column, a "1" represents the national estimate, a "2" represents the census region estimates, a "3" represents the state-level estimates, a "4" represents the substate/aggregate region-level estimates included in the maps, and a "5" represents the other substate/aggregate region-level estimates that are not included in the maps.
Source: SAMHSA, Center for Behavioral Health Statistics and Quality, National Survey on Drug Use and Health, 2012, 2013, and 2014.

Description obtain from official SAMHSA source


<br \><br \><br \>

### Definition
**Rate of Suicidal thoughts:** Number of people out of 100 who have reported having seriously considered suicide in the last 12 months

<br \><br \><br \>

In [2]:
DS_suicidal_thoughts = pd.read_excel('D:\\Datasets\\2012-2014_Substate_SAE_Table_26.xlsx')

DS_national = DS_suicidal_thoughts.loc[DS_suicidal_thoughts['Map Group'] == 1]

DS_CensusRegion = DS_suicidal_thoughts.loc[DS_suicidal_thoughts['Map Group'] == 2]

DS_Statelevel = DS_suicidal_thoughts.loc[DS_suicidal_thoughts['Map Group'] == 3]

DS_SubstateLevel = DS_suicidal_thoughts.loc[DS_suicidal_thoughts['Map Group'] == 4]

DS_Other = DS_suicidal_thoughts.loc[DS_suicidal_thoughts['Map Group'] == 5]

In [31]:
DS_CensusRegion

,Order,State,Substate Region,Small Area Estimate,95% CI (Lower),95% CI (Upper),Map Group
1,2,Northeast,Northeast,0.038257,0.035640,0.041057,2
2,3,Midwest,Midwest,0.040398,0.038152,0.042769,2
3,4,South,South,0.038261,0.036098,0.040547,2
4,5,West,West,0.039860,0.037087,0.042831,2


In [38]:
"""
    Getting some preliminary inferences from the data. Here we have shown the state with the 
    largest rate of suicidal thoughts per capita

"""

DS_Statelevel.loc[DS_Statelevel['Small \nArea Estimate'] == max(DS_Statelevel['Small \nArea Estimate'])]

,Order,State,Substate Region,Small Area Estimate,95% CI (Lower),95% CI (Upper),Map Group
411,412,Utah,Utah,0.048478,0.041583,0.056449,3


In [4]:
import shapefile
# read the shapefile
reader = shapefile.Reader("cb_2017_us_region_500k\\cb_2017_us_region_500k.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 

# write the GeoJSON file
from json import dumps
geojson = open("US_regions.json", "w")
geojson.write(dumps({"type": "FeatureCollection",\
"features": buffer}, indent=2) + "\n")
geojson.close()

### Plotting rate of suicidal thoughts by state

Description

In [16]:
"""



"""


from branca.colormap import linear

colormap = linear.YlOrRd_09.scale(
    DS_Statelevel['Small \nArea Estimate'].min()*100,
    DS_Statelevel['Small \nArea Estimate'].max()*100)

#print(colormap(1))



colormap

In [33]:
"""



"""

geo_json_data = json.load(open('D:\\Datasets\\folium_example_data\\data\\us-states.json'))

In [34]:
"""


"""
folium_map = folium.Map(tiles="CartoDB dark_matter")
#folium.GeoJson(geo_json_data).add_to(folium_map)


In [35]:
DS_Statelevel_dict = DS_Statelevel.set_index('State')['Small \nArea Estimate']*100

In [36]:
folium.GeoJson(
    geo_json_data,
    name='Suicidal Thoughts by State',
    style_function=lambda feature: {
        'fillColor': colormap(DS_Statelevel_dict[feature['properties']['name']]),
        'color': 'black',
        'weight': 1,
        #'dashArray': '1, 1',
        'fillOpacity': 0.5,
    }
).add_to(folium_map)

colormap.caption = 'Rate of Suicidal thoughts %'
colormap.add_to(folium_map)

In [37]:
folium_map

In [39]:
folium_map.save('suicidal_thoughts_by_state_2012-2014.html')